## 准备数据

In [25]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [26]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [28]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        # parameters are defined in the function train_one_step: W1, b1, W2, b2
        # input layer size: 28*28
        # output layer size: 10(categories)
        # hide layer size: 128
        
#         self.W1 = tf.Variable(shape=[28*28, 128], 
#                               initial_value=tf.random.uniform(shape=[28*28, 128]))
#         self.b1 = tf.Variable(shape=[128],
#                               initial_value=tf.zeros(128))
#         self.W2 = tf.Variable(shape=[128, 10],
#                               initial_value=tf.random.uniform(shape=[128, 10]))
#         self.b2 = tf.Variable(shape=[10], 
#                               initial_value=tf.zeros(10))
        
        self.W1 = tf.Variable(shape=[28*28, 128], 
                              initial_value=tf.random.uniform(shape=[28*28, 128],minval=-0.1, maxval=0.1))
        self.b1 = tf.Variable(shape=[128], initial_value=tf.zeros(128))
        self.W2 = tf.Variable(shape=[128, 10], 
                              initial_value=tf.random.uniform(shape=[128, 10],minval=-0.1, maxval=0.1))
        self.b2 = tf.Variable(shape=[10], initial_value=tf.zeros(10))
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        flat_x = tf.reshape(x, shape=[-1, 28*28])  # to match matrix multiply
#         h = tf.sigmoid(tf.matmul(flat_x, self.W1) + self.b1) # sigmoid
#         h = tf.nn.relu(tf.matmul(flat_x, self.W1) + self.b1) # ReLU
        h = tf.tanh(tf.matmul(flat_x, self.W1) + self.b1) # tanh
        logits = tf.matmul(h, self.W2) + self.b2
        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [29]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [30]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.3664753 ; accuracy 0.09245
epoch 1 : loss 2.3594947 ; accuracy 0.09723333
epoch 2 : loss 2.3525739 ; accuracy 0.102016665
epoch 3 : loss 2.345711 ; accuracy 0.10733333
epoch 4 : loss 2.3389041 ; accuracy 0.11245
epoch 5 : loss 2.332152 ; accuracy 0.1177
epoch 6 : loss 2.325452 ; accuracy 0.12285
epoch 7 : loss 2.3188035 ; accuracy 0.1278
epoch 8 : loss 2.3122048 ; accuracy 0.13301666
epoch 9 : loss 2.3056545 ; accuracy 0.13805
epoch 10 : loss 2.299151 ; accuracy 0.14341667
epoch 11 : loss 2.2926927 ; accuracy 0.14878333
epoch 12 : loss 2.2862792 ; accuracy 0.15433334
epoch 13 : loss 2.2799087 ; accuracy 0.15995
epoch 14 : loss 2.2735798 ; accuracy 0.16546667
epoch 15 : loss 2.2672923 ; accuracy 0.17085
epoch 16 : loss 2.2610438 ; accuracy 0.17625
epoch 17 : loss 2.2548342 ; accuracy 0.18251666
epoch 18 : loss 2.2486622 ; accuracy 0.18905
epoch 19 : loss 2.242527 ; accuracy 0.19465
epoch 20 : loss 2.2364273 ; accuracy 0.20095
epoch 21 : loss 2.2303624 ; accuracy 0.20726